In [ ]:
# !pip install pororo -q

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

from datetime import datetime, timedelta
import time 
from tqdm import trange

#### 🅰제품 리뷰 수집

In [3]:
code = [
        ['2010266907', '메디웰 구수한맛'      ], 
        ['1164812508', '메디웰 당뇨식 플러스' ], 
        ['1164855482', '메디웰 식이섬유'      ], 
        ['2011956710', '메디웰 당뇨식 플러스' ], 
        ]

In [5]:

date_start = (datetime.today()+ timedelta(hours=9) - timedelta(days=30))
date_end   = (datetime.today()+ timedelta(hours=9) - timedelta(days=0))

for goodsCode, item in code:
    url  = 'http://item.gmarket.co.kr/Review/Premium' # 접속할 사이트주소 또는 IP주소를 입력한다 
    data = {'goodsCode' : goodsCode, 'pageNo' : '1', 'sort' : '1', 'totalPage' : '1'}          # 요청할 데이터
    response = requests.post(url=url, params=data)
    bsObj = BeautifulSoup(response.text, "html.parser")
    for i in range(len(bsObj('td', {'class' : 'comment-content'}))):
        USER_temp = bsObj('dl', {'class' : 'writer-info'})[i]('dd')[0].text
        REG_DTM_temp = bsObj('dl', {'class' : 'writer-info'})[i]('dd')[1].text.replace('.', '-')
        REG_DTM_temp = datetime.strptime(REG_DTM_temp, '%Y-%m-%d')
        REVIEW_1 = bsObj('td', {'class' : 'comment-content'})[i]('p', {'class': 'comment-tit'})[0].text
        REVIEW_2 = bsObj('td', {'class' : 'comment-content'})[i]('p', {'class': 'con'})[0].text
        REVIEW_temp = REVIEW_1 + ' ' + REVIEW_2 
        PRODUCT_temp = item
        # if date_start <= REG_DTM_temp <=date_end:
        print(USER_temp, REG_DTM_temp.strftime('%Y-%m-%d'), REVIEW_temp)

cso**** 2023-01-17 좋아요 허전할때 간식으로 좋아요~~
jjb*** 2023-01-03 좋습니다 좋습니다 그리고 무엇보다 중요한 것이 선택입니다
hed*** 2023-01-03 좋아요 빠르게 잘 왔네요 좋아요
bbg***** 2022-12-15 좋아요 괜찮다고 해서 주문합니다
aud** 2022-12-01 잘샀어요 좋은 가격에 잘 샀어요!
toz**** 2023-01-29 맛있겠죠 처음 구매하네요. 좋으면 계속 구매하고 싶어요
ppo*** 2022-11-11 엄마가 당이 높아져서 구매했어요 드시니 바로 당조절되네요
d83***** 2022-06-21 보충제 보충제 찾아보다가 당뇨 환자식이라 주문해봅니다
nav***** 2022-04-22 생각보다 짭조름 하네요. 당료식이라 밍밍하고 맛없을줄 알았는데 고소하고 맛있어요.
내가 어쩌다가 당뇨가 걸려서...에효 ㅠ
lwj** 2022-04-20 굳굳 아버지 드릴려고 샀어요
pyh**** 2023-01-14 선물 드렸어요 당뇨가 있으신 지인분께 상품평이 좋아서 드셔보시라라고 보내드렸습니다.~
car****** 2022-12-03 늘 주문하는 ... 당뇨 있으신 부모님께 계속  사드리는 제품
rhd****** 2022-10-31 좋아요 쿠폰으로 저렴하게 구매했어요
rhd****** 2022-10-31 좋아요 부모님 당뇨식으로 사드렸어요
did******* 2022-10-11 좋은상품 만족합니다. 좋은상품 만족합니다.


In [ ]:
i = 0
UESR = bsObj('dl', {'class' : 'writer-info'})[i]('dd')[0].text
REG_DTM = bsObj('dl', {'class' : 'writer-info'})[i]('dd')[1].text
REVIEW_1 = bsObj('td', {'class' : 'comment-content'})[i]('p', {'class': 'comment-tit'})[0].text
REVIEW_2 = bsObj('td', {'class' : 'comment-content'})[i]('p', {'class': 'con'})[0].text
REVIEW = REVIEW_1 + ' ' + REVIEW_2 
PRODUCT = '어메이징 오트 오리지널'

In [ ]:
UESR, REG_DTM, REVIEW, PRODUCT

('KMS****', '2022.04.26', '크림파스타소스 간편하게 맛있게 먹을수있다해서 주문했어요', '어메이징 오트 오리지널')

In [ ]:
df = pd.DataFrame()
df['USER'] = USER
df['REG_DTM'] = REG_DTM
df['REG_DTM'] =df.REG_DTM.str.replace('.', '-')
df['REVIEW'] = REVIEW
df['PRODUCT'] = PRODUCT

NameError: ignored

#### 🆗제품 리뷰 수집(API)

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
date_start = (datetime.today()+ timedelta(hours=9) - timedelta(days=1)).replace(minute=0, hour=0, second=0, microsecond=0)
date_end   = (datetime.today()+ timedelta(hours=9) - timedelta(days=0)).replace(minute=0, hour=0, second=0, microsecond=0)
date_start, date_end  

(datetime.datetime(2022, 7, 13, 0, 0), datetime.datetime(2022, 7, 14, 0, 0))

In [ ]:
# main.py
import pandas as pd
from datetime import datetime, timedelta
import requests
import time 
import re
from google.cloud import bigquery
from bs4 import BeautifulSoup

# client = bigquery.Client(project = 'boxwood-chassis-356100')
# dataset_id = 'review'
# table_id = 'review_all'
# table_ref = client.dataset(dataset_id).table(table_id)
# Table = client.get_table(table_ref)  # API request

def gmarket_review(t1):
    date_start = (datetime.today() + timedelta(hours=9) - timedelta(1)).replace(minute=0, hour=0, second=0, microsecond=0)
    date_end   = (datetime.today() + timedelta(hours=9) - timedelta(0)).replace(minute=0, hour=0, second=0, microsecond=0)
    code = [
            ['2216945469', '어메이징 오트 오리지널 190ml 48팩'], 
            ['2237111945', '어메이징 오트 오리지널 190ml 48팩'], #스마일배송샵
            ['2236243295', '어메이징 오트 오리지널 190ml 24팩'], #스마일배송샵
            ['2216944085', '어메이징 오트 오리지널 190ml 24팩'], 
            ['2236241590', '어메이징 오트 언스위트 190ml 48팩'], #스마일배송샵
            ['2216949482', '어메이징 오트 언스위트 190ml 48팩'], 
            
            # ['2258037532', '앱솔루트 명작 2FL 분유 1단계 800g 3개'], 
            # ['2258038193', '앱솔루트 명작 2FL 분유 2단계 800g 3개'], 
            # ['2258036116', '앱솔루트 명작 2FL 분유 3단계 800g 3개'], 

            ['1787478689', '상하목장 슬로우키친 토마토 파스타 소스 170g 6팩'], 
            ['1787491082', '상하목장 슬로우키친 로제 파스타 소스 155g 6팩'  ], 
            ['1787481753', '상하목장 슬로우키친 크림 파스타 소스 145g 6팩'  ], 
            
            ['2143308748', '상하키친 로제치킨 브리또 130g 6개'           ], 
            ['2143307486', '상하키친 불고기마요 브리또 130g 6개'         ], 
            ['2143303019', '상하키친 칠리치즈 포테이토 브리또 130g 6개'  ], 
            ['2143309905', '상하키친 브리또 130g 칠리 2개+불고기 2개+로제 2개'  ],

            ['1881987933', '상하목장 유기농 사과딸기케일 주스 125ml 24팩'    ],
            ['1881993919', '상하목장 유기농 사과오렌지케일 주스 125ml 24팩'  ],
            ['1905038578', '상하 유기농주스 사과딸기케일+오렌지케일 총 48팩' ],

            ]
    for goodsCode, item in code:
        url  = 'http://item.gmarket.co.kr/Review/Premium' # 접속할 사이트주소 또는 IP주소를 입력한다 
        for j in range(1,4):
            data = {'goodsCode' : goodsCode, 'pageNo' : str(j), 'sort' : '1', 'totalPage' : '3'}          # 요청할 데이터
            response = requests.post(url=url, params=data)
            bsObj = BeautifulSoup(response.text, "html.parser")
            for i in range(len(bsObj('td', {'class' : 'comment-content'}))):
                user = bsObj('dl', {'class' : 'writer-info'})[i]('dd')[0].text
                DATE = bsObj('dl', {'class' : 'writer-info'})[i]('dd')[1].text.replace('.', '-')
                DATE = datetime.strptime(DATE, '%Y-%m-%d')
                REVIEW_1 = bsObj('td', {'class' : 'comment-content'})[i]('p', {'class': 'comment-tit'})[0].text
                REVIEW_2 = bsObj('td', {'class' : 'comment-content'})[i]('p', {'class': 'con'})[0].text.replace('\n', ' ')
                review_text = REVIEW_1 + ' ' + REVIEW_2 
                product = item
                if date_start <= DATE < date_end:
                    rows_to_insert = [(user, 'G마켓', product, review_text, '0', DATE.strftime('%Y-%m-%d %H:%M:%S'), '리뷰')]
                    # client.insert_rows(Table, rows_to_insert)
                    print(rows_to_insert)
                else:
                    pass
    # return print('OK')  

In [ ]:
gmarket_review('')

[('ben*****', 'G마켓', '어메이징 오트 오리지널 190ml 48팩', '맛나고 좋아요. 살찔걱정 없고 부담없이 자주 마셔요.', 0, '2022-07-13 00:00:00', '리뷰')]
[('onl*******', 'G마켓', '어메이징 오트 오리지널 190ml 24팩', '처음 먹어봐요~ 약간 두유맛이 나면서 담백하고 먹으니 은근 든든해서 좋네요~!', 0, '2022-07-13 00:00:00', '리뷰')]
[('min*****', 'G마켓', '상하목장 슬로우키친 로제 파스타 소스 155g 6팩', '굿 맛있어요 아이들이 너믄 좋아해요 소스중 젤 맛있는듯', 0, '2022-07-13 00:00:00', '리뷰')]
[('wls*******', 'G마켓', '상하목장 유기농 사과딸기케일 주스 125ml 24팩', '상하 안전하게 도착했습니다 좀 늦게', 0, '2022-07-13 00:00:00', '리뷰')]
[('seo*******', 'G마켓', '상하목장 유기농 사과딸기케일 주스 125ml 24팩', '그럭저럭 잘 구입 했네요 저렴하게 잘 구입 했네요', 0, '2022-07-13 00:00:00', '리뷰')]
[('hap****', 'G마켓', '상하목장 유기농 사과딸기케일 주스 125ml 24팩', '유기농주스 많이 달지 않아서 맛있어요', 0, '2022-07-13 00:00:00', '리뷰')]
[('min****', 'G마켓', '상하목장 유기농 사과딸기케일 주스 125ml 24팩', '잘 마실께요 잘 마실께요 잘 받았어요', 0, '2022-07-13 00:00:00', '리뷰')]
[('jun*****', 'G마켓', '상하목장 유기농 사과딸기케일 주스 125ml 24팩', '양이 진짜 작네요 양이 진짜 미니네요 ㅎㅎ 성인은 2개는 마셔야할거같네요 유기농에 저지방우유 건강한맛이네요', 0, '2022-07-13 00:00:00', '리뷰')]
[('chj****', 'G마켓', '상하목장 유기농 사과딸기케일 주스 1